In [181]:
import requests
from dataclasses import dataclass
from typing import List, Optional, Dict
import conf
import os
from datetime import datetime, date, timedelta
# from parsing_old import parse_movie_data, process_cinemas
import json


datatime

In [182]:
dict_cinema = conf.CINEMAS
url_root = conf.URL_ROOT

In [183]:
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR')

'fr_FR'

In [184]:
seances = {'C2954': {'cinemaName': 'Mk2 bibliothèque',
                    'showtimes': ['2025-02-01T14:40:00', '2025-02-01T17:00:00', 
                                  '2025-02-01T19:15:00', '2025-02-01T22:00:00', 
                                  '2025-02-02T09:50:00', '2025-02-02T12:15:00', 
                                  '2025-02-02T14:40:00', '2025-02-02T17:00:00', 
                                  '2025-02-02T19:15:00', '2025-02-02T22:00:00', 
                                  '2025-02-03T11:10:00', '2025-02-03T14:40:00', 
                                  '2025-02-03T17:00:00', '2025-02-03T19:15:00', 
                                  '2025-02-03T22:00:00', '2025-02-04T11:10:00', 
                                  '2025-02-04T14:40:00', '2025-02-04T17:00:00', 
                                  '2025-02-04T19:15:00', '2025-02-04T22:00:00']}, 
            'C0026': {'cinemaName': 'UGC ciné Bercy', 
                      'showtimes': ['2025-02-01T10:20:00', '2025-02-01T12:40:00', 
                                    '2025-02-01T15:00:00', '2025-02-01T17:20:00', 
                                    '2025-02-01T19:40:00', '2025-02-01T22:00:00', 
                                    '2025-02-02T10:20:00', '2025-02-02T12:40:00', 
                                    '2025-02-02T15:00:00', '2025-02-02T17:20:00', 
                                    '2025-02-02T19:40:00', '2025-02-02T22:00:00', 
                                    '2025-02-03T10:20:00', '2025-02-03T12:40:00', 
                                    '2025-02-03T15:00:00', '2025-02-03T17:20:00', 
                                    '2025-02-03T19:40:00', '2025-02-03T22:00:00', 
                                    '2025-02-04T10:20:00', '2025-02-04T12:40:00', 
                                    '2025-02-04T15:00:00', '2025-02-04T17:20:00', 
                                    '2025-02-04T19:40:00', '2025-02-04T22:00:00']}}

In [188]:
show_dict = {}
for key, values in seances.items():
    cine = seances.get(key).get('cinemaName')
    for showtime in seances.get(key).get('showtimes'):
        readable_date = datetime.strptime(showtime, '%Y-%m-%dT%H:%M:%S').strftime('%A %m-%d')
        readable_time = datetime.strptime(showtime, '%Y-%m-%dT%H:%M:%S').strftime('%H:%M:%S')
        if not readable_date in show_dict:
            show_dict[readable_date] = {}
        if not cine in show_dict[readable_date]:
            show_dict[readable_date][cine] = [readable_time]
        else:
            show_dict[readable_date][cine].append(readable_time)
        

show_dict

{'Samedi 02-01': {'Mk2 bibliothèque': ['14:40:00',
   '17:00:00',
   '19:15:00',
   '22:00:00'],
  'UGC ciné Bercy': ['10:20:00',
   '12:40:00',
   '15:00:00',
   '17:20:00',
   '19:40:00',
   '22:00:00']},
 'Dimanche 02-02': {'Mk2 bibliothèque': ['09:50:00',
   '12:15:00',
   '14:40:00',
   '17:00:00',
   '19:15:00',
   '22:00:00'],
  'UGC ciné Bercy': ['10:20:00',
   '12:40:00',
   '15:00:00',
   '17:20:00',
   '19:40:00',
   '22:00:00']},
 'Lundi 02-03': {'Mk2 bibliothèque': ['11:10:00',
   '14:40:00',
   '17:00:00',
   '19:15:00',
   '22:00:00'],
  'UGC ciné Bercy': ['10:20:00',
   '12:40:00',
   '15:00:00',
   '17:20:00',
   '19:40:00',
   '22:00:00']},
 'Mardi 02-04': {'Mk2 bibliothèque': ['11:10:00',
   '14:40:00',
   '17:00:00',
   '19:15:00',
   '22:00:00'],
  'UGC ciné Bercy': ['10:20:00',
   '12:40:00',
   '15:00:00',
   '17:20:00',
   '19:40:00',
   '22:00:00']}}

In [186]:
show_dict

{'Samedi 02-01': {'Mk2 bibliothèque': ['14:40:00',
   '17:00:00',
   '19:15:00',
   '22:00:00'],
  'UGC ciné Bercy': ['10:20:00',
   '12:40:00',
   '15:00:00',
   '17:20:00',
   '19:40:00',
   '22:00:00']},
 'Dimanche 02-02': {'Mk2 bibliothèque': ['09:50:00',
   '12:15:00',
   '14:40:00',
   '17:00:00',
   '19:15:00',
   '22:00:00'],
  'UGC ciné Bercy': ['10:20:00',
   '12:40:00',
   '15:00:00',
   '17:20:00',
   '19:40:00',
   '22:00:00']},
 'Lundi 02-03': {'Mk2 bibliothèque': ['11:10:00',
   '14:40:00',
   '17:00:00',
   '19:15:00',
   '22:00:00'],
  'UGC ciné Bercy': ['10:20:00',
   '12:40:00',
   '15:00:00',
   '17:20:00',
   '19:40:00',
   '22:00:00']},
 'Mardi 02-04': {'Mk2 bibliothèque': ['11:10:00',
   '14:40:00',
   '17:00:00',
   '19:15:00',
   '22:00:00'],
  'UGC ciné Bercy': ['10:20:00',
   '12:40:00',
   '15:00:00',
   '17:20:00',
   '19:40:00',
   '22:00:00']}}

In [187]:
datetime.strptime('2025-01-31T11:05:00', '%Y-%m-%dT%H:%M:%S').strftime('%A %m-%d')

'Vendredi 01-31'